In [1]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
import shared_utils

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics

from siuba import *
import pandas as pd
import geopandas as gpd

import datetime as dt

import importlib
import rt_analysis as rt
import rt_filter_map_plot

from IPython.display import display, Markdown, Latex, HTML
import json

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

In [2]:
itp_id = 4 ## AC Transit

In [3]:
import numpy as np

In [4]:
date_range = np.arange('2022-04-30', '2022-05-09', dtype='datetime64[D]')

In [5]:
date_range

array(['2022-04-30', '2022-05-01', '2022-05-02', '2022-05-03',
       '2022-05-04', '2022-05-05', '2022-05-06', '2022-05-07',
       '2022-05-08'], dtype='datetime64[D]')

# Generate analysis data for all dates

In [6]:
pbar = tqdm()

0it [00:00, ?it/s]

In [7]:
# for date in date_range:
#     date = date.astype(dt.date)
#     try:
#         rt_analysis = rt.OperatorDayAnalysis(itp_id, date, pbar)
#         rt_analysis.export_views_gcs()
#         print(f'complete for date: {date}')
#     except:
#         print(f'failed for date: {date}')
#         continue

In [8]:
# for date in date_range:
#     date = date.astype(dt.date)
#     try:
#         rt_day = rt_filter_map_plot.from_gcs(itp_id, date)
#         print(f'complete for date: {date}')
#     except:
#         print(f'failed for date: {date}')
#         continue

In [9]:
rt_day = rt_filter_map_plot.from_gcs(4, dt.date(2022, 5, 1))

found parquet


In [12]:
rt_day.stop_delay_view.dropna(subset=['stop_id']) >> head(3)

,shape_meters,stop_id,stop_name,geometry,shape_id,trip_key,trip_id,stop_sequence,arrival_time,route_id,route_short_name,direction_id,actual_time,delay_seconds
0,574.272090,1346,Walnut Av & Guardino Dr,POINT (-173895.624 -49034.139),shp-217-10,-5.354689e+18,13428010,2.0,2022-05-01 15:31:28,217,217,0,2022-05-01 15:32:21.890113,53.0
1,943.266663,1341,Walnut Av & Gallaudet Dr,POINT (-173601.966 -48812.414),shp-217-10,-5.354689e+18,13428010,3.0,2022-05-01 15:32:14,217,217,0,2022-05-01 15:33:11.180694,57.0
2,1204.489885,1394,Walnut Av & Walnut Circle,POINT (-173406.632 -48740.636),shp-217-10,-5.354689e+18,13428010,4.0,2022-05-01 15:32:51,217,217,0,2022-05-01 15:33:53.019003,62.0


In [13]:
corridor = gpd.read_file('./CMCP_StudyAreaBoundary.geojson').to_crs(shared_utils.geography_utils.CA_NAD83Albers)

In [14]:
to_clip = rt_day.stop_delay_view.drop_duplicates(subset=['shape_id', 'stop_sequence']).dropna(subset=['stop_id'])

In [15]:
clipped = to_clip.clip(corridor)

In [16]:
shape_seqences = (rt_day.stop_delay_view.dropna(subset=['stop_id']) >> distinct(_.shape_id, _.stop_sequence) >> arrange(_.shape_id, _.stop_sequence))

In [17]:
## ex. stop 7 is first within bounding box
(shape_seqences >> filter(_.shape_id == 'shp-10-09') >> filter(_.stop_sequence < 7)).stop_sequence.max()

6.0

In [18]:
## ex. stop 12 is last within bounding box
(shape_seqences >> filter(_.shape_id == 'shp-10-09') >> filter(_.stop_sequence > 12)).stop_sequence.min()

13.0

In [19]:
## no stop returns np.nan
(shape_seqences >> filter(_.shape_id == 'shp-10-09') >> filter(_.stop_sequence < 2)).stop_sequence.max()

nan

In [20]:
clipped_min = (clipped >> filter(_.shape_id == 'shp-10-09')).stop_sequence.min()

In [21]:
(clipped >> filter(_.shape_id == 'shp-10-09')).stop_sequence.max()

33.0

In [24]:
shape_sequence_filter = {}
for shape_id in clipped.shape_id.unique():
# for shape_id in ['shp-10-09']:
    shape_sequence_filter[shape_id] = {}
    clipped_min = (clipped >> filter(_.shape_id == shape_id)).stop_sequence.min()
    clipped_max = (clipped >> filter(_.shape_id == shape_id)).stop_sequence.max()
    this_shape = shape_seqences >> filter(_.shape_id == shape_id)
    filter_min = (this_shape >> filter(_.stop_sequence < clipped_min)).stop_sequence.max()
    filter_max = (this_shape >> filter(_.stop_sequence > clipped_max)).stop_sequence.min()
    shape_sequence_filter[shape_id]['min'] = max(0, filter_min)
    shape_sequence_filter[shape_id]['max'] = filter_max if not np.isnan(filter_max) else clipped_max

In [38]:
shape_sequence_filter

{'shp-41-54': {'min': 0, 'max': 26.0},
 'shp-41-04': {'min': 12.0, 'max': 35.0},
 'shp-60-07': {'min': 0, 'max': 29.0},
 'shp-86-01': {'min': 0, 'max': 39.0},
 'shp-86-51': {'min': 0, 'max': 39.0},
 'shp-60-56': {'min': 7.0, 'max': 37.0},
 'shp-99-09': {'min': 0, 'max': 17.0},
 'shp-801-01': {'min': 36.0, 'max': 84.0},
 'shp-99-10': {'min': 37.0, 'max': 52.0},
 'shp-93-09': {'min': 9.0, 'max': 53.0},
 'shp-56-53': {'min': 0, 'max': 6.0},
 'shp-93-56': {'min': 0, 'max': 47.0},
 'shp-56-03': {'min': 43.0, 'max': 47.0},
 'shp-34-56': {'min': 9.0, 'max': 56.0},
 'shp-86-52': {'min': 0, 'max': 6.0},
 'shp-86-02': {'min': 7.0, 'max': 9.0},
 'shp-34-07': {'min': 0, 'max': 49.0},
 'shp-28-10': {'min': 0, 'max': 65.0},
 'shp-10-09': {'min': 0, 'max': 33.0},
 'shp-95-07': {'min': 0, 'max': 7.0},
 'shp-10-10': {'min': 0, 'max': 35.0},
 'shp-95-53': {'min': 16.0, 'max': 21.0},
 'shp-28-59': {'min': 0, 'max': 64.0},
 'shp-97-57': {'min': 0, 'max': 5.0},
 'shp-35-11': {'min': 0, 'max': 47.0},
 'shp-

In [39]:
# gdf = rt_day.stop_delay_view >> filter(_.shape_id == 'shp-40-19', _.stop_sequence >= 8, _.stop_sequence <= 34)
# gdf = gdf[['stop_id', 'geometry']]

# pd.concat([corridor, gdf]).explore(tiles = "CartoDB positron")

# Algorithm

* Clip stops to corridor
* Get first and last stops in corridor for each shape_id, then find 1 stop before and 1 stop after for each
* Filter stop_delay_view based on those stops
* Assert that more than 50% of filtered stops within corridor, if not then drop shape (remove shapes starting/ending in corridor but mostly running outside)

In [143]:
# can use this as # of stops in corr...
clipped >> count(_.shape_id)

,shape_id,n
0,shp-10-09,32
1,shp-10-10,34
2,shp-12-56,2
3,shp-14-15,19
4,shp-14-59,19
...,...,...
62,shp-98-52,13
63,shp-99-09,15
64,shp-99-10,15
65,shp-O-03,1


In [42]:
fn = lambda x: x.shape_id in (shape_sequence_filter.keys()) and x.stop_sequence >= shape_sequence_filter[x.shape_id]['min'] and x.stop_sequence <= shape_sequence_filter[x.shape_id]['max']
rt_day.stop_delay_view['corridor'] = rt_day.stop_delay_view.apply(fn, axis=1)

In [44]:
(rt_day.stop_delay_view >> filter(_.corridor) >> select(_.stop_id, _.geometry, _.stop_sequence).explore(tiles = "CartoDB positron")

TypeError: Object of type Timestamp is not JSON serializable